In [ ]:
import numpy as np
from PIL import Image
import io
from pathlib import Path
import os
import torch
import matplotlib.pyplot as plt
import random


In [8]:
# On utilise %pip pour cibler le bon environnement Python
# On ajoute --user pour éviter l'erreur "Accès refusé"

print("⏳ Installation de PyTorch (version CUDA)...")
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122 --user

print("⏳ Installation des outils d'image...")
%pip install opencv-python pillow numpy matplotlib tqdm --user

print("⏳ Installation de Transformers et Hugging Face...")
%pip install transformers huggingface-hub peft accelerate --user

print("✅ Installation terminée. VEUILLEZ REDÉMARRER LE KERNEL.")

⏳ Installation de PyTorch (version CUDA)...
Note: you may need to restart the kernel to use updated packages.Looking in indexes: https://download.pytorch.org/whl/cu122

⏳ Installation des outils d'image...


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ----------- ---------------------------- 11.3/39.0 MB 72.2 MB/s eta 0:00:01
   ----------------------------------- ---- 34.9/39.0 MB 97.7 MB/s eta 0:00:01
   ---------------------------------------- 39.0/39.0 MB 79.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
⏳ Installation de Transformers et Hugging Face...
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------  11.8/12.0 MB 79.6 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 62.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   --------------------------------------- 566.1/566.1 kB 35.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 70.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/556.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
print("✅ Import réussi !")

✅ Import réussi !


In [ ]:
import numpy as np

print("=" * 50)
print("DETAILED CHANNEL ANALYSIS")
print("=" * 50)

# Check each channel separately
for i in range(3):
    channel = depth_data[:, :, i]
    valid_count = (~np.isnan(channel)).sum()
    nan_count = np.isnan(channel).sum()

    print(f"\nChannel {i}:")
    print(f"  Valid values: {valid_count} ({valid_count / channel.size * 100:.2f}%)")
    print(f"  NaN values: {nan_count} ({nan_count / channel.size * 100:.2f}%)")

    if valid_count > 0:
        print(f"  Min (ignoring NaN): {np.nanmin(channel):.2f}")
        print(f"  Max (ignoring NaN): {np.nanmax(channel):.2f}")
        print(f"  Mean (ignoring NaN): {np.nanmean(channel):.2f}")

# Overall statistics
total_pixels = depth_data.shape[0] * depth_data.shape[1]
total_values = depth_data.size
valid_pixels_count = (~np.isnan(depth_data)).any(axis=2).sum()

print(f"\n{'=' * 50}")
print("OVERALL STATISTICS")
print("=" * 50)
print(f"Total pixels: {total_pixels}")
print(f"Pixels with at least one valid value: {valid_pixels_count} ({valid_pixels_count/total_pixels*100:.1f}%)")
print(f"Total values (across 3 channels): {total_values}")
print(f"Total valid values: {(~np.isnan(depth_data)).sum()}")

# Try to find and display some valid values if they exist
print(f"\n{'=' * 50}")
print("SEARCHING FOR VALID DATA...")
print("=" * 50)

valid_mask = ~np.isnan(depth_data)
if valid_mask.any():
    # Find pixels with valid data
    valid_pixels = np.any(valid_mask, axis=2)
    valid_coords = np.argwhere(valid_pixels)

    if len(valid_coords) > 0:
        print(f"Found {len(valid_coords)} pixels with valid data!")
        print(f"\nFirst 3 valid pixels:")
        for idx in valid_coords[:3]:
            y, x = idx
            print(f"  Position ({y}, {x}): {depth_data[y, x]}")
else:
    print("⚠️  WARNING: NO VALID DATA FOUND IN ENTIRE FILE!")
    print("\nPossible issues:")
    print("  1. File is corrupted")
    print("  2. Wrong file exported from Zivid")
    print("  3. Data needs special loading procedure")

DETAILED CHANNEL ANALYSIS

Channel 0:
  Valid values: 1627140 (69.75%)
  NaN values: 705660 (30.25%)
  Min (ignoring NaN): -1269.60
  Max (ignoring NaN): 804.97
  Mean (ignoring NaN): -86.09

Channel 1:
  Valid values: 1627140 (69.75%)
  NaN values: 705660 (30.25%)
  Min (ignoring NaN): -615.58
  Max (ignoring NaN): 670.23
  Mean (ignoring NaN): 56.33

Channel 2:
  Valid values: 1627140 (69.75%)
  NaN values: 705660 (30.25%)
  Min (ignoring NaN): 1310.65
  Max (ignoring NaN): 2961.60
  Mean (ignoring NaN): 1755.40

OVERALL STATISTICS
Total pixels: 2332800
Pixels with at least one valid value: 1627140 (69.8%)
Total values (across 3 channels): 6998400
Total valid values: 4881420

SEARCHING FOR VALID DATA...
Found 1627140 pixels with valid data!

First 3 valid pixels:
  Position (31, 6): [-734.60004 -425.95316 1340.2417 ]
  Position (31, 7): [-728.9061  -423.09607 1335.3439 ]
  Position (31, 9): [-725.3841  -421.93024 1331.5236 ]


Preparation de l'env


In [2]:
def load_pretrained_model(model_name="depth-anything/Depth-Anything-V2-Small-hf"):
    """
    Charge le modèle Depth Anything V2 pré-entraîné et son processeur d'images
    """
    print(f"📥 Chargement du modèle: {model_name}")

    image_processor = AutoImageProcessor.from_pretrained(model_name)
    base_model = AutoModelForDepthEstimation.from_pretrained(model_name)

    print(f"✅ Modèle chargé avec succès")
    return base_model, image_processor

In [3]:
base_model, image_processor = load_pretrained_model()

📥 Chargement du modèle: depth-anything/Depth-Anything-V2-Small-hf


preprocessor_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

C:\Users\abchikhi\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abchikhi\.cache\huggingface\hub\models--depth-anything--Depth-Anything-V2-Small-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Using a slow image processor as `use_fast` is unset and a slow processor was 

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/99.2M [00:00<?, ?B/s]

✅ Modèle chargé avec succès


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = base_model.to(device)

In [8]:
# Count parameters
total_params = sum(p.numel() for p in base_model.parameters())
trainable_params = sum(p.numel() for p in base_model.parameters() if p.requires_grad)

print(f"📊 Total parameters: {total_params:,}")
print(f"📊 Trainable parameters: {trainable_params:,}")

📊 Total parameters: 24,785,089
📊 Trainable parameters: 24,785,089


In [10]:
# ===================================================
# INSPECT MODEL ARCHITECTURE
# ===================================================
print("🔍 Inspecting model architecture...")
print("\n" + "="*60)

# Print all module names
print("MODEL STRUCTURE:")
for name, module in base_model.named_modules():
    print(f"  {name}: {module.__class__.__name__}")

print("\n" + "="*60)
print("TRAINABLE LAYERS (before LoRA):")
for name, param in base_model.named_parameters():
    if param.requires_grad:
        print(f"  {name}: {param.shape}")

🔍 Inspecting model architecture...

MODEL STRUCTURE:
  : DepthAnythingForDepthEstimation
  backbone: Dinov2Backbone
  backbone.embeddings: Dinov2Embeddings
  backbone.embeddings.patch_embeddings: Dinov2PatchEmbeddings
  backbone.embeddings.patch_embeddings.projection: Conv2d
  backbone.embeddings.dropout: Dropout
  backbone.encoder: Dinov2Encoder
  backbone.encoder.layer: ModuleList
  backbone.encoder.layer.0: Dinov2Layer
  backbone.encoder.layer.0.norm1: LayerNorm
  backbone.encoder.layer.0.attention: Dinov2Attention
  backbone.encoder.layer.0.attention.attention: Dinov2SelfAttention
  backbone.encoder.layer.0.attention.attention.query: Linear
  backbone.encoder.layer.0.attention.attention.key: Linear
  backbone.encoder.layer.0.attention.attention.value: Linear
  backbone.encoder.layer.0.attention.output: Dinov2SelfOutput
  backbone.encoder.layer.0.attention.output.dense: Linear
  backbone.encoder.layer.0.attention.output.dropout: Dropout
  backbone.encoder.layer.0.layer_scale1: Dinov

PREPARATION  des données

In [15]:

path_images = r"P:\images"
path_depths = r"P:\depth"

In [17]:
print(f"📂 Dossier Images visé : {path_images}")
print(f"📂 Dossier Depth visé  : {path_depths}")

# Vérification que les dossiers existent
if os.path.exists(path_images) and os.path.exists(path_depths):
    print("\n✅ Les dossiers existent !")
    
    # Compter les fichiers pour être sûr
    nb_img = len([f for f in os.listdir(path_images) if f.endswith(('.png', '.jpg'))])
    nb_npy = len([f for f in os.listdir(path_depths) if f.endswith('.npy')])
    
    print(f"   -> Trouvé {nb_img} images")
    print(f"   -> Trouvé {nb_npy} fichiers XYZ (.npy)")
else:
    print("\n❌ ERREUR : Un des chemins est introuvable.")
    print("   Vérifie l'orthographe ou copie le chemin absolu.")

📂 Dossier Images visé : P:\images
📂 Dossier Depth visé  : P:\depth

✅ Les dossiers existent !
   -> Trouvé 58 images
   -> Trouvé 58 fichiers XYZ (.npy)


In [16]:
# 2. COMPTAGE ET LISTING
# ==========================================
print("🔍 Démarrage de la vérification...")

# On liste et on trie pour garantir l'ordre
# On ne garde que les fichiers (pas les dossiers cachés)
files_img = sorted([f for f in os.listdir(path_images) if f.endswith(('.png', '.jpg'))])
files_npy = sorted([f for f in os.listdir(path_depths) if f.endswith('.npy')])

num_img = len(files_img)
num_npy = len(files_npy)

print(f"📊 Bilan comptable :")
print(f"   - Images (.png/.jpg) : {num_img}")
print(f"   - Depths (.npy)      : {num_npy}")

🔍 Démarrage de la vérification...
📊 Bilan comptable :
   - Images (.png/.jpg) : 58
   - Depths (.npy)      : 58


In [18]:
# ==========================================
# 3. VÉRIFICATION DE L'ALIGNEMENT
# ==========================================
if num_img != num_npy:
    print(f"❌ ATTENTION : Le nombre de fichiers ne correspond pas ! (Différence : {abs(num_img - num_npy)})")
else:
    print("✅ Le nombre de fichiers est identique.")

# Vérification des correspondances de noms (optionnel mais recommandé)
# On vérifie si image_01_color.png a bien son équivalent image_01_rawDepth.npy
mismatches = []
for img_name in files_img:
    # On recrée le nom attendu du fichier depth (selon la logique Zivid vue dans ton notebook)
    # Ex: "scan_color.png" -> "scan_rawDepth.npy"
    expected_depth_name = img_name.replace("_color.png", "_rawDepth.npy").replace(".png", ".npy")
    
    # Si tes fichiers n'ont pas "_color", ajuste cette ligne (ex: replace(".png", ".npy"))
    
    if expected_depth_name not in files_npy:
        mismatches.append((img_name, expected_depth_name))

if len(mismatches) > 0:
    print(f"⚠️ ATTENTION : {len(mismatches)} paires ne correspondent pas au niveau du nom !")
    print(f"   Exemple de manquant : {mismatches[0]}")
else:
    print("✅ Tous les noms de fichiers semblent correspondre (logique color <-> rawDepth).")

✅ Le nombre de fichiers est identique.
✅ Tous les noms de fichiers semblent correspondre (logique color <-> rawDepth).


In [ ]:
# ==========================================
# 4. TEST D'EXTRACTION DU Z (Le fameux Z)
# ==========================================

if num_npy > 0:
    print("\n🧪 Test d'extraction du canal Z sur un fichier aléatoire...")
    
    # Prendre un fichier au hasard
    random_npy = random.choice(files_npy)
    full_path = os.path.join(path_depths, random_npy)
    
    try:
        # Chargement
        data = np.load(full_path) # Shape (H, W, 3) normalement
        
        print(f"   Fichier : {random_npy}")
        print(f"   Shape originale : {data.shape}")
        
        if len(data.shape) == 3 and data.shape[2] == 3:
            # EXTRACTION DU Z (Canal index 2)
            Z_channel = data[:, :, 2]
            
            # Vérif statistiques
            print(f"   ✅ Extraction Z réussie ! Shape finale : {Z_channel.shape}")
            print(f"   Val Min (avec NaN) : {np.nanmin(Z_channel)}")
            print(f"   Val Max (avec NaN) : {np.nanmax(Z_channel)}")
            
            # Vérification des NaN
            nan_count = np.isnan(Z_channel).sum()
            print(f"   Nombre de pixels NaN (vides) : {nan_count} ({(nan_count/Z_channel.size)*100:.1f}%)")
        else:
            print("❌ Erreur : Le fichier .npy n'a pas 3 canaux (XYZ). Vérifie le format.")
            
    except Exception as e:
        print(f"❌ Erreur lors du chargement : {e}")


🧪 Test d'extraction du canal Z sur un fichier aléatoire...
   Fichier : 21-12-03-19-05-46_Zivid_acquisition_rawDepth.npy
   Shape originale : (1200, 1944, 3)
   ✅ Extraction Z réussie ! Shape finale : (1200, 1944)
   Val Min (avec NaN) : 851.3021240234375
   Val Max (avec NaN) : 2249.957275390625
   Nombre de pixels NaN (vides) : 652630 (28.0%)


In [21]:
filename = "21-12-03-19-05-46_Zivid_acquisition_rawDepth.npy"
full_path = os.path.join(path_depths, filename)

# --- EXTRACTION ---
# 1. Chargement de la donnée (1200, 1944, 3)
data_xyz = np.load(full_path)

# 2. Extraction de la carte de profondeur Z (1200, 1944)
Z_channel = data_xyz[:, :, 2] 

# 3. Traitement des NaN (pour s'assurer qu'on peut lire)
Z_clean = np.nan_to_num(Z_channel, nan=0.0)

# --- LECTURE DE LA VALEUR D'UN PIXEL ---
pixel_row = 600
pixel_col = 972

# Accéder à la valeur Z du pixel (600, 972)
profondeur_pixel = Z_clean[pixel_row, pixel_col]

print(f"Le fichier chargé est de forme : {Z_channel.shape}")
print(f"La valeur de Profondeur (Z) au pixel [{pixel_row}, {pixel_col}] est :")
print(f"-> **{profondeur_pixel:.2f} mm**")

Le fichier chargé est de forme : (1200, 1944)
La valeur de Profondeur (Z) au pixel [600, 972] est :
-> **1365.02 mm**
